Dataset :- https://www.kaggle.com/rmisra/news-category-dataset

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [3]:
import pandas as pd
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [4]:
import numpy as np
df['TF'] = np.where(df['category']  == 'CRIME', 1, np.where(df['category']=='WEIRD NEWS',1,np.where(df['category']=='WOMEN',1,0)))
#df['TF'] = np.where(df['category']=='WEIRD NEWS' ,1,0)
#df['TF'] = np.where(df['category']=='WOMEN',1,0)

In [5]:
sentences=df['headline']
labels=df['TF']

In [6]:
df.shape

(200853, 7)

In [7]:
training_sentences = sentences[0:18419]
testing_sentences = sentences[18419:]
training_labels = labels[0:18419]
testing_labels = labels[18419:]

In [9]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [10]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 16,289
Trainable params: 16,289
Non-trainable params: 0
_________________________________________________________________


In [13]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)

Epoch 1/30
576/576 [==============================] - 10s 18ms/step - loss: 0.3231 - accuracy: 0.9257 - val_loss: 0.1872 - val_accuracy: 0.9544
Epoch 2/30
576/576 [==============================] - 9s 16ms/step - loss: 0.2443 - accuracy: 0.9326 - val_loss: 0.1857 - val_accuracy: 0.9544
Epoch 3/30
576/576 [==============================] - 9s 16ms/step - loss: 0.2406 - accuracy: 0.9326 - val_loss: 0.1856 - val_accuracy: 0.9544
Epoch 4/30
576/576 [==============================] - 10s 17ms/step - loss: 0.2349 - accuracy: 0.9326 - val_loss: 0.1850 - val_accuracy: 0.9544
Epoch 5/30
576/576 [==============================] - 9s 16ms/step - loss: 0.2261 - accuracy: 0.9326 - val_loss: 0.1890 - val_accuracy: 0.9544
Epoch 6/30
576/576 [==============================] - 10s 17ms/step - loss: 0.2148 - accuracy: 0.9329 - val_loss: 0.1817 - val_accuracy: 0.9546
Epoch 7/30
576/576 [==============================] - 10s 17ms/step - loss: 0.2026 - accuracy: 0.9345 - val_loss: 0.1780 - val_accuracy: 0.

In [14]:
key = tokenizer.texts_to_sequences(["Ram killed the innocent"])
key = pad_sequences(key)

In [25]:
loss, accuracy = model.evaluate(training_padded, training_labels, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(testing_padded, testing_labels, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9389
Testing Accuracy:  0.9531


In [15]:
model.predict(key)

array([[0.95639515]], dtype=float32)

In [16]:
from GoogleNews import GoogleNews

In [17]:
gn = GoogleNews('India')
gn.set_lang('en')
gn.search('Chennai Theft')

In [18]:
a=gn.results()

In [19]:
for i in a:
    txt=[str(i['title'])]
    txt=np.array(txt)
    key = tokenizer.texts_to_sequences(txt)
    key = pad_sequences(key)
    if txt != ['']:
        print(txt," - > ",model.predict(key))

['Gang stealing Royal Enfield bikes in Chennai arrested']  - >  [[0.9979756]]
['100 kg gold ‘vanishes into thin air’ from CBI custody']  - >  [[0.00892904]]
['Chennai: This ‘Bullet’ cannot be dodged']  - >  [[0.98456514]]
['Projectors worth over ₹15 lakh stolen from Anna University']  - >  [[0.00053397]]
['Gold jewellery stolen from PWD official’s home in Thanjavur']  - >  [[0.7598971]]
["Chennai Brothers Held for Stealing Goats to Fund Father's Movie Starring Them in Lead Roles"]  - >  [[8.052699e-07]]
['New norms reduce cab fare, increase safety']  - >  [[0.00044468]]
['Chairman, manager of private hospital held in Coimbatore']  - >  [[0.01386097]]
['Tamil Nadu: ‘Intoxicated’ man drives off with government bus as driver checks its tyres']  - >  [[8.4828185e-05]]
['Chennai businessman complains of theft of ₹45 lakh from his home']  - >  [[0.91410005]]


In [20]:
model.save('model_news1.h5')

In [21]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)